In [16]:
from radar_class.armor_predictor import YoloV5
import cv2
import torch
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.augmentations import (Albumentations, augment_hsv, classify_albumentations, classify_transforms, copy_paste,
                                        letterbox, mixup, random_perspective)
import numpy as np
from yolov5.utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, colorstr, cv2,
                                  increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
# net = YoloV5()
net =  DetectMultiBackend("/home/cooper/Documents/RM2023-Deep-Radar/yolov5/runs/train/car_30011/weights/best.pt",
                                        device=torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"), data='yolov5/data/car.yaml')
net1 = DetectMultiBackend('/home/cooper/Documents/RM2023-Deep-Radar/yolov5/armor.pt',device=torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"), data='yolov5/data/armor.yaml')
img0= cv2.imread('/home/cooper/Documents/RM2023-Deep-Radar/dataset/combined/test/images/field_000405.jpg')

img_preds, car_locations = [], []
#show image
img = img0.copy()
img = letterbox(img, [640, 640], stride=32, auto=False)[
                0]  # padded resize
img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
img = np.ascontiguousarray(img)  # contiguous
img = torch.from_numpy(img).to(torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")).float()
img = img / 255
if len(img.shape) == 3:
    img = img[None]  # expand for batch dim
ori_shape = (img0.shape[0], img0.shape[1], img0.shape[2])

pred= net(img)
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)
pred[0][:, :4] = scale_boxes(
    img.shape[2:], pred[0][:, :4], ori_shape).round()

car_pred_per_img = []
armor_pred_per_img = []
car_bbox_pre_img = []

ori_car_pred_per_img = []
ori_armor_pred_per_img = []
armor_num_per_img = []
if pred[0].shape[0] == 0:
    img_preds.append([])
    car_locations.append([None, None])

for i in range(pred[0].shape[0]):
    if (pred[0][i][5] == 0):
        ori_car_pred_per_img.append(pred[0][i])

        car_x1 = pred[0][i][0].cpu().numpy().astype(np.uint16) 
        car_y1 = pred[0][i][1].cpu().numpy().astype(np.uint16)
        car_x2 = pred[0][i][2].cpu().numpy().astype(np.uint16)
        car_y2 = pred[0][i][3].cpu().numpy().astype(np.uint16)
        car_acc = pred[0][i][4].cpu().numpy().astype(np.uint16)

        # predict the number of armor
        # crop as x1+1/4*width, x2-1/4*width for the armor_area to remove the LEDs for armor

        ori_car_area = img0[car_y1:car_y2,
                                car_x1:car_x2]
        
        car_area = ori_car_area.copy()

        cv2.imwrite('test_armor.png', ori_car_area*255)
        cv2.imwrite('test_ori_armor.png', ori_car_area)

        # input is row: 20 col: 28 format: CV_8UC1
        ori_car_area_grey = (cv2.cvtColor(car_area, cv2.COLOR_BGR2GRAY) /
                255.0).astype(np.float32)

        car_area_img = letterbox(car_area, [640, 640], stride=32, auto=False)[
            0]  # padded resize
        car_area_img = car_area_img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
        car_area_img = np.ascontiguousarray(car_area_img)  # contiguous
        car_area_img = torch.from_numpy(car_area_img).to(torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")).float()
        car_area_img = car_area_img / 255
        if len(car_area_img.shape) == 3:
            car_area_img = car_area_img[None]  # expand for batch dim

        # torch.Size([1, 3, 448, 640])
        pred_armor = net1(car_area_img)
        # output of the pred: shape (b, N, 6)
        # x1, y1, x2, y2, acc, class

        pred_armor = non_max_suppression(pred_armor, conf_thres=0.25, iou_thres=0.45)
        
        pred_armor[0][:, :4] = scale_boxes(
            car_area_img.shape[2:], pred_armor[0][:, :4], ori_shape).round()
        

        for k in range(pred_armor[0].shape[0]):
            armor_num_per_img.append(pred_armor[0][k][5])
            ori_armor_pred_per_img.append(pred_armor[0][k])
            armor_x1 = pred_armor[0][k][0].cpu().numpy().astype(np.uint16)
            armor_y1 = pred_armor[0][k][1].cpu().numpy().astype(np.uint16)
            armor_x2 = pred_armor[0][k][2].cpu().numpy().astype(np.uint16)
            armor_y2 = pred_armor[0][k][3].cpu().numpy().astype(np.uint16)
            armor_acc = pred_armor[0][k][4].cpu().numpy().astype(np.uint16)

#draw the bbox of armor
            cv2.namedWindow('image', cv2.WINDOW_NORMAL)
            cv2.rectangle(img0, (782, 1882),
                        (1497, 2160), (0, 0, 255), 2)
            #show the image in proper size
            cv2.imshow('image', img0)
            cv2.waitKey(0)
                        

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Fusing layers... 
Model summary: 157 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
